In [1]:
import pandas as pd

In [2]:
mult_reg_results, _, _  = pd.read_pickle('/auto/data2/oelmas/RSA_Results/Naive/Video/correlation_hamming_regression_results.pkl')

In [3]:
mult_reg_results

,experiment_type,stimuli_type,electrode_region,model_name,time,beta_value,t_value,p_value,lower_conf_interval,upper_conf_interval
513,naive,video,central,action_category_hamming,-200,0.052211,0.349135,0.727263,-0.242226,0.346648
4117,naive,video,frontal,flowModel_10_frames_correlation,-200,0.150793,0.566093,0.571806,-0.373670,0.675256
4116,naive,video,frontal,biological_appearance_hamming,-200,-0.017748,-0.626133,0.531763,-0.073557,0.038061
4115,naive,video,frontal,agent_hamming,-200,0.015726,0.261568,0.793856,-0.102650,0.134103
4114,naive,video,frontal,action_target_hamming,-200,0.036257,1.558411,0.120320,-0.009550,0.082063
...,...,...,...,...,...,...,...,...,...,...
3397,naive,video,central,flowModel_10_frames_correlation,598,0.054490,0.175657,0.860696,-0.556269,0.665249
3398,naive,video,central,movement_nature_hamming,598,-0.046126,-1.619490,0.106522,-0.102204,0.009952
3399,naive,video,central,gabor_video_10_frames_correlation,598,0.017425,0.121838,0.903119,-0.264165,0.299016
6997,naive,video,frontal,flowModel_10_frames_correlation,598,0.081706,0.258372,0.796319,-0.540921,0.704332


In [4]:
VIF =  pd.read_csv('/auto/data2/oelmas/RSA_Results/Naive/Video/parietal_VIF_Results.csv')

In [5]:
VIF

,Unnamed: 0,VIF Factor,features
0,0,14.154372,action_category_hamming
1,1,2.814802,action_target_hamming
2,2,16.203330,agent_hamming
3,3,5.402288,biological_appearance_hamming
4,4,921.422867,flowModel_10_frames_correlation
5,5,4.021932,movement_nature_hamming
6,6,78.927001,gabor_video_10_frames_correlation
7,7,609.975974,avg_pixel_intensity_model_hamming
8,8,63.349774,pixel_wise_motion_model_hamming


In [6]:
from statsmodels.sandbox.stats.multicomp import multipletests

In [7]:
sig_idx = multipletests(mult_reg_results["p_value"], alpha=0.05, method='bonferroni')[0]
mult_reg_results['significant'] = sig_idx

In [14]:
#er_idx = mult_reg_results.electrode_region=="parietal" 
#mod_idx = mult_reg_results.model_name=="movement_nature_hamming" 
#df = mult_reg_results[er_idx & mod_idx]

#df[df.beta_value == df.beta_value.max()]

In [15]:
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as po

In [16]:
avg_sbj_results, eeg_dist, model_dist  = pd.read_pickle('/auto/data2/oelmas/RSA_Results/Naive/Video/correlation_hamming_avg_subjects_results.pkl')

In [17]:
rdm_df = avg_sbj_results
model_name = 'movement_nature_hamming'
electrode_region = 'parietal'
constant = 'naive'
variable = 'video'

In [18]:
df_to_plot = rdm_df[  (rdm_df["model_name"] == model_name) 
                    & (rdm_df["electrode_region"] == electrode_region)
                    & (rdm_df["experiment_type"] == constant) 
                    & (rdm_df["stimulus_type"] == variable)]

In [19]:
er_idx = mult_reg_results.electrode_region=="parietal" 
mod_idx = mult_reg_results.model_name=="movement_nature_hamming" 
reg_df = mult_reg_results[er_idx & mod_idx]

In [30]:
min(reg_df.loc[reg_df['significant']].time)

64

In [35]:
fig = go.Figure()
 
# Plots the kendall tau curve
fig.add_trace(go.Scatter(x=df_to_plot["time"],
                        y=df_to_plot["kendall_tau"],
                        mode='lines'))


fig.add_shape(
        # Rectangle reference to the plot
            type="rect",
            xref="x",
            yref="paper",
            x0=min(reg_df.loc[reg_df['significant']].time),
            y0=0,
            x1=max(reg_df.loc[reg_df['significant']].time),
            y1=1,
            opacity=0.7,
            layer="below",
            line_width=0,
            fillcolor="LightSalmon",
        )

fig.update_yaxes(range=[-0.2, 0.6])
fig.update_yaxes(title_text="Kendall-tau")
fig.update_xaxes(title_text="Time ms (relative to stimulus onset)")
po.plot(fig, filename = "outputs/TEST.html", auto_open=False)

'outputs/TEST.html'

In [21]:
mult_reg_results.model_name.unique()

array(['action_category_hamming', 'flowModel_10_frames_correlation',
       'biological_appearance_hamming', 'agent_hamming',
       'action_target_hamming', 'pixel_wise_motion_model_hamming',
       'avg_pixel_intensity_model_hamming',
       'gabor_video_10_frames_correlation', 'movement_nature_hamming'],
      dtype=object)

In [22]:
#############################################################
#############################################################


In [30]:
er_idx = mult_reg_results.electrode_region=="parietal" 
ex_idx = mult_reg_results.experiment_type=='naive'
st_idx = mult_reg_results.stimuli_type=='video'

for model in mult_reg_results.model_name.unique():
    mod_idx = mult_reg_results.model_name==model
    
    reg_df = mult_reg_results[ex_idx & st_idx & er_idx & mod_idx]
    sig_idx = multipletests(reg_df["p_value"], alpha=0.05, method='bonferroni')[0]
    sig_time = reg_df[sig_idx].time



In [34]:
sig_idx = multipletests(mult_reg_results["p_value"], alpha=0.05, method='bonferroni')[0]
mult_reg_results['significant'] = sig_idx

In [64]:
reg_df = mult_reg_results[ex_idx & st_idx & er_idx]


fig = px.scatter(reg_df.loc[reg_df['significant']], 
                 x="time", y="model_name",  color="model_name")
plotly.offline.plot(fig, filename = "outputs/SCATTER.html", auto_open=False)

'outputs/SCATTER.html'